### Coursera Assignment "Segmenting and Clustering Neighborhoods in Toronto"

#### 1. Scrape the Wikipedia page *https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M* to obtain the data that is in the table of postal codes, transform these data into a pandas dataframe, and modify the dataframe so that it meets the required specifications

1.1 Import libraries

In [20]:
import folium
import requests
import folium
import requests 
import json 

import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np 

from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from sklearn.cluster import KMeans

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

1.2 Get the table from the website by using the *pd.read_html* function, it is the first element that is returned

In [21]:
tor_web = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

tor_neighbor = tor_web[0]
tor_neighbor.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


1.3 Rename the columns of the dataframe

In [22]:
tor_neighbor.rename(columns={"Postcode": "PostalCode", "Neighbourhood": "Neighborhood"},inplace=True)
tor_neighbor.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


1.4 Drop cells that do not have an assigned borough.

In [23]:
tor_neighbor.drop(tor_neighbor.loc[tor_neighbor['Borough']=="Not assigned"].index, inplace=True)
tor_neighbor.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


1.5 Combine neighborhoods that have the same borough

In [24]:
tor_neighbor = tor_neighbor.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
tor_neighbor.head(50)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


1.6 Assign *neighborhood* the same value as *borough* if it is not assigned

In [25]:
tor_neighbor.loc[tor_neighbor['Neighborhood'] =='Not assigned','Neighborhood'] = tor_neighbor['Borough']
tor_neighbor.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


**1.7 Print number of rows in dataframe using the *shape* method - required in assignment**

In [26]:
tor_neighbor.shape[0]

103

#### 2. Add geographical coordinates by reading the *Geospatial_Coordinates.csv* into a dataframe and merging this dataframe  with the *tor_neighbor* dataframe

In [27]:
geo = pd.read_csv("Geospatial_Coordinates.csv")
geo.rename(columns={'Postal Code':'PostalCode'},inplace=True)

geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [28]:
tor_neighbor_merge = pd.merge(tor_neighbor, geo, on='PostalCode')
tor_neighbor_merge.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [29]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(tor_neighbor_merge['Borough'].unique()),
        tor_neighbor_merge.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


#### 3. Explore and cluster the neighborhoods in Toronto. 

3.1 Get the coordinates of Toronto

In [30]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


3.2 Create a map of Toronto

In [31]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(tor_neighbor_merge['Latitude'], tor_neighbor_merge['Longitude'], tor_neighbor_merge['Borough'], tor_neighbor_merge['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare Credentials and Version

In [32]:
CLIENT_ID = 'RMLS3DLWXKID2GGBHCHWRTMPAFW0VZPE33PS3P00SJJKY3VB' # your Foursquare ID
CLIENT_SECRET = 'EUKE2PXBL2GZXVHNGWXCP5W1MSCFUW5OBEFWQUQR2YOTO4K0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

3.3 Define function to get the nearyby venues

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
           
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
requests.get(url).json()

{'meta': {'code': 200, 'requestId': '5d341963bbf1bd00233d2da5'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.711248304499996,
    'lng': -79.58784096674265},
   'sw': {'lat': 43.70224829549999, 'lng': -79.60026783325736}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5273f6f311d2137549494e29',
       'name': 'Economy Rent A Car',
       'location': {'lat': 43.708471,
        'lng': -79.589943,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.708471,
          'lng': -79.589943}],


3.4 Get venues in the neigborhoods

In [35]:
toronto_venues = getNearbyVenues(names=tor_neighbor_merge['Neighborhood'],
                                   latitudes=tor_neighbor_merge['Latitude'],
                                   longitudes=tor_neighbor_merge['Longitude'])

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

3.5 Check the size of the resulting dataframe

In [36]:
print(toronto_venues.shape)
toronto_venues.head()

(2255, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


3.6 Check how many venues were returned for each neighborhood

In [37]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,56,56,56,56,56,56


3.7 As suggested, I perform the analysis only with boroughs that contain the word "Toronto" and replicate the analysis of the New York City data performed in the course. 

In [38]:
tor_ven_tor = toronto_venues[toronto_venues['Neighborhood'].str.contains('Toronto')]
tor_ven_tor.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
259,"CFB Toronto, Downsview East",43.737473,-79.464763,Toronto Downsview Airport (YZD),43.738883,-79.470111,Airport
260,"CFB Toronto, Downsview East",43.737473,-79.464763,First Class Realty Ltd,43.737133,-79.463298,Electronics Store
261,"CFB Toronto, Downsview East",43.737473,-79.464763,Ancaster Park,43.734706,-79.464777,Park
354,East Toronto,43.685347,-79.338106,Pizza Pizza,43.683065,-79.333846,Pizza Place
355,East Toronto,43.685347,-79.338106,Aldwych Park,43.684901,-79.341091,Park


3.8 Check out the dimensions of the *tor_ven_tor* dataframe

In [39]:
tor_ven_tor.shape

(275, 7)

3.9 Find out how many unique categories can be curated from all the returned venues

In [40]:
print('There are {} uniques categories.'.format(len(tor_ven_tor['Venue Category'].unique())))

There are 100 uniques categories.


3.10 Analyze each neighborhood

In [82]:
# One hot encoding to transform column 'Venu Category' 
tor_ven_tor_onehot=pd.get_dummies(tor_ven_tor[['Venue Category']], prefix="", prefix_sep="")

# Remove the 'Neighborhood' column in the dataset
tor_ven_tor_onehot.drop(['Neighborhood'],axis=1,inplace=True)

# Add Neighborhood' column from tor_ven_tor dataset
tor_ven_tor_onehot['Neighborhood'] = tor_ven_tor['Neighborhood'] 

# Move newly created neighborhood column to the first column
cols = tor_ven_tor_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Neighborhood')))
tor_ven_tor_onehot = tor_ven_tor_onehot[cols]

# Look at recoded dataset
tor_ven_tor_onehot.head()

,Neighborhood,Airport,American Restaurant,Aquarium,Art Gallery,Asian Restaurant,Bakery,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Store,Bistro,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Fast Food Restaurant,Flower Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,General Travel,Greek Restaurant,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Lake,Liquor Store,Lounge,Mexican Restaurant,Miscellaneous Shop,Monument / Landmark,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Park,Performing Arts Venue,Pharmacy,Pizza Place,Plaza,Poutine Place,Pub,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skating Rink,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Tea Room,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Yoga Studio
259,"CFB Toronto, Downsview East",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
260,"CFB Toronto, Downsview East",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
261,"CFB Toronto, Downsview East",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
354,East Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
355,East Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


3.11 Examine the size of each neighborhood

In [83]:
tor_ven_tor_onehot.shape

(275, 100)

3.12 Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [84]:
tor_ven_tor_grouped = tor_ven_tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_ven_tor_grouped

,Neighborhood,Airport,American Restaurant,Aquarium,Art Gallery,Asian Restaurant,Bakery,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Store,Bistro,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Fast Food Restaurant,Flower Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,General Travel,Greek Restaurant,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Lake,Liquor Store,Lounge,Mexican Restaurant,Miscellaneous Shop,Monument / Landmark,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Park,Performing Arts Venue,Pharmacy,Pizza Place,Plaza,Poutine Place,Pub,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skating Rink,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Tea Room,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Yoga Studio
0,"CFB Toronto, Downsview East",0.333333,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.00,0.000000,0.00,0.00,0.333333,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.333333,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00
1,"Design Exchange, Toronto Dominion Centre",0.000000,0.030000,0.00,0.01,0.01,0.020000,0.030000,0.00,0.01,0.020000,0.000000,0.00,0.010000,0.01,0.00,0.00,0.02,0.080000,0.010000,0.00,0.01,0.140000,0.000000,0.000000,0.000000,0.01,0.0,0.00,0.03,0.000000,0.00,0.00,0.000000,0.00,0.010000,0.000000,0.01,0.010000,0.010000,0.00,0.03,0.01,0.01,0.030000,0.01,0.00,0.00,0.06,0.01,0.01,0.00,0.040000,0.020000,0.00,0.000000,0.02,0.00,0.00,0.00,0.01,0.00,0.01,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.020000,0.01,0.000000,0.010000,0.00,0.040000,0.00,0.01,0.00,0.010000,0.00,0.020000,0.01,0.00,0.00,0.01,0.01,0.01,0.00,0.02,0.00,0.000000,0.01,0.01,0.010000,0.00,0.01,0.01,0.000000,0.01,0.00
2,East Toronto,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.200000,0.000000,0.000000,0.000000,0.00,0.2,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.400000,0.00,0.000000,0.200000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00
3,"Harbord, University of Toronto",0.000000,0.000000,0.00,0.00,0.00,0.060606,0.060606,0.00,0.00,0.030303,0.030303,0.00,0.060606,0.00,0.00,0.00,0.00,0.121212,0.030303,0.00,0.00,0.030303,0.030303,0.030303,0.030303,0.00,0.0,0.00,0.00,0.030303,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.030303,0.000000,0.00,0.00,0.00,0.00,0.030303,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.030303,0.060606,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.030303,0.030303,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.030303,0.030303,0.00,0.060606,0.00,0.00,0.00,0.030303,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.030303,0.00,0.00,0.

3.13 Let's confirm the new size

In [85]:
tor_ven_tor_grouped.shape

(7, 100)

3.14 Print each neighborhood along with the top 5 most common venues

In [86]:
num_top_venues = 5

for hood in tor_ven_tor_grouped['Neighborhood']:
    temp = tor_ven_tor_grouped[tor_ven_tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

                 venue  freq
0              Airport  0.33
1    Electronics Store  0.33
2                 Park  0.33
3         Noodle House  0.00
4  Rental Car Location  0.00


                venue  freq
0         Coffee Shop  0.14
1                Café  0.08
2               Hotel  0.06
3  Italian Restaurant  0.04
4          Restaurant  0.04


               venue  freq
0               Park   0.4
1        Coffee Shop   0.2
2        Pizza Place   0.2
3  Convenience Store   0.2
4            Airport   0.0


                 venue  freq
0                 Café  0.12
1           Restaurant  0.06
2               Bakery  0.06
3                  Bar  0.06
4  Japanese Restaurant  0.06


                venue  freq
0         Coffee Shop  0.11
1               Hotel  0.05
2            Aquarium  0.05
3  Italian Restaurant  0.04
4                Café  0.04


                venue  freq
0          Restaurant  0.07
1  Seafood Restaurant  0.07
2            Pharmacy  0.07
3         Pizza Place  0.07
4   

3.15 Define a function to sort venues in descending order

In [89]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

3.16 Create the new dataframe and display the top 10 venues for each neighborhood.

In [90]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_ven_tor_grouped['Neighborhood']

for ind in np.arange(tor_ven_tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_ven_tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"CFB Toronto, Downsview East",Airport,Park,Electronics Store,Flower Shop,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store
1,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Italian Restaurant,Restaurant,Deli / Bodega,Gym,American Restaurant,Bar,Gastropub
2,East Toronto,Park,Convenience Store,Coffee Shop,Pizza Place,Fast Food Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store
3,"Harbord, University of Toronto",Café,Bakery,Japanese Restaurant,Restaurant,Bar,Bookstore,College Arts Building,Coffee Shop,College Gym,Poutine Place
4,"Harbourfront East, Toronto Islands, Union Station",Coffee Shop,Aquarium,Hotel,Italian Restaurant,Café,Sporting Goods Shop,Scenic Lookout,Pizza Place,Brewery,Bakery


#### 4. Run *k*-means to cluster the neighborhood into 5 clusters and examine the clusters**

4.1 Run *k*-means clustering

In [92]:
# set number of clusters
kclusters = 5

# drop the 'Neighborhood' column
toronto_grouped_clustering = tor_ven_tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 2, 0, 0, 4, 3], dtype=int32)

4.2 Add clustering labels to the *neighborhoods_venues_sorted* dataframe

In [93]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,"CFB Toronto, Downsview East",Airport,Park,Electronics Store,Flower Shop,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store
1,0,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Italian Restaurant,Restaurant,Deli / Bodega,Gym,American Restaurant,Bar,Gastropub
2,2,East Toronto,Park,Convenience Store,Coffee Shop,Pizza Place,Fast Food Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store
3,0,"Harbord, University of Toronto",Café,Bakery,Japanese Restaurant,Restaurant,Bar,Bookstore,College Arts Building,Coffee Shop,College Gym,Poutine Place
4,0,"Harbourfront East, Toronto Islands, Union Station",Coffee Shop,Aquarium,Hotel,Italian Restaurant,Café,Sporting Goods Shop,Scenic Lookout,Pizza Place,Brewery,Bakery


4.3 Merge the *neighborhoods_venues_sorted* and the *tor_neighbor_merge* dataframes, i.e. create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood 

In [94]:
#rename the *tor_neighbor_merge* dataframe
toronto_merged = tor_ven_tor

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
259,"CFB Toronto, Downsview East",43.737473,-79.464763,Toronto Downsview Airport (YZD),43.738883,-79.470111,Airport,1,Airport,Park,Electronics Store,Flower Shop,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store
260,"CFB Toronto, Downsview East",43.737473,-79.464763,First Class Realty Ltd,43.737133,-79.463298,Electronics Store,1,Airport,Park,Electronics Store,Flower Shop,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store
261,"CFB Toronto, Downsview East",43.737473,-79.464763,Ancaster Park,43.734706,-79.464777,Park,1,Airport,Park,Electronics Store,Flower Shop,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store
354,East Toronto,43.685347,-79.338106,Pizza Pizza,43.683065,-79.333846,Pizza Place,2,Park,Convenience Store,Coffee Shop,Pizza Place,Fast Food Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store
355,East Toronto,43.685347,-79.338106,Aldwych Park,43.684901,-79.341091,Park,2,Park,Convenience Store,Coffee Shop,Pizza Place,Fast Food Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store


4.4 Visualize clusters

In [95]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Neighborhood Latitude'], toronto_merged['Neighborhood Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

4.5 Look at the different clusters

Cluster 1 - Coffee Shops, Aquariums, Hotels

In [96]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1173,43.640816,-79.380688,Neighborhood,0,Coffee Shop,Aquarium,Hotel,Italian Restaurant,Café,Sporting Goods Shop,Scenic Lookout,Pizza Place,Brewery,Bakery
1174,43.640816,-79.384279,Park,0,Coffee Shop,Aquarium,Hotel,Italian Restaurant,Café,Sporting Goods Shop,Scenic Lookout,Pizza Place,Brewery,Bakery
1175,43.640816,-79.382081,Salad Place,0,Coffee Shop,Aquarium,Hotel,Italian Restaurant,Café,Sporting Goods Shop,Scenic Lookout,Pizza Place,Brewery,Bakery
1176,43.640816,-79.380197,Bakery,0,Coffee Shop,Aquarium,Hotel,Italian Restaurant,Café,Sporting Goods Shop,Scenic Lookout,Pizza Place,Brewery,Bakery
1177,43.640816,-79.381393,Supermarket,0,Coffee Shop,Aquarium,Hotel,Italian Restaurant,Café,Sporting Goods Shop,Scenic Lookout,Pizza Place,Brewery,Bakery
1178,43.640816,-79.380184,Sporting Goods Shop,0,Coffee Shop,Aquarium,Hotel,Italian Restaurant,Café,Sporting Goods Shop,Scenic Lookout,Pizza Place,Brewery,Bakery
1179,43.640816,-79.379606,Bubble Tea Shop,0,Coffee Shop,Aquarium,Hotel,Italian Restaurant,Café,Sporting Goods Shop,Scenic Lookout,Pizza Place,Brewery,Bakery
1180,43.640816,-79.379665,Lake,0,Coffee Shop,Aquarium,Hotel,Italian Restaurant,Café,Sporting Goods Shop,Scenic Lookout,Pizza Place,Brewery,Bakery
1181,43.640816,-79.383749,Café,0,Coffee Shop,Aquarium,Hotel,Italian Restaurant,Café,Sporting Goods Shop,Scenic Lookout,Pizza Place,Brewery,Bakery
1182,43.640816,-79.383190,Performing Arts Venue,0,Coffee Shop,Aquarium,Hotel,Italian Restaurant,Café,Sporting Goods Shop,Scenic Lookout,Pizza Place,Brewery,Bakery


Cluster 2 - Airport, Park, Electronics

In [97]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
259,43.737473,-79.470111,Airport,1,Airport,Park,Electronics Store,Flower Shop,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store
260,43.737473,-79.463298,Electronics Store,1,Airport,Park,Electronics Store,Flower Shop,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store
261,43.737473,-79.464777,Park,1,Airport,Park,Electronics Store,Flower Shop,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store


Cluster 3 - Parks, Convenience Stores, and Coffee Shops

In [98]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
354,43.685347,-79.333846,Pizza Place,2,Park,Convenience Store,Coffee Shop,Pizza Place,Fast Food Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store
355,43.685347,-79.341091,Park,2,Park,Convenience Store,Coffee Shop,Pizza Place,Fast Food Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store
356,43.685347,-79.335007,Park,2,Park,Convenience Store,Coffee Shop,Pizza Place,Fast Food Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store
357,43.685347,-79.335007,Convenience Store,2,Park,Convenience Store,Coffee Shop,Pizza Place,Fast Food Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store
358,43.685347,-79.333274,Coffee Shop,2,Park,Convenience Store,Coffee Shop,Pizza Place,Fast Food Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store


Cluster 4- Coffee Shops, Clothing Stores, Spas

In [99]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
470,43.715383,-79.400109,Yoga Studio,3,Coffee Shop,Clothing Store,Spa,Furniture / Home Store,Fast Food Restaurant,Mexican Restaurant,Miscellaneous Shop,Diner,Dessert Shop,Park
471,43.715383,-79.400091,Diner,3,Coffee Shop,Clothing Store,Spa,Furniture / Home Store,Fast Food Restaurant,Mexican Restaurant,Miscellaneous Shop,Diner,Dessert Shop,Park
472,43.715383,-79.400303,Coffee Shop,3,Coffee Shop,Clothing Store,Spa,Furniture / Home Store,Fast Food Restaurant,Mexican Restaurant,Miscellaneous Shop,Diner,Dessert Shop,Park
473,43.715383,-79.400000,Mexican Restaurant,3,Coffee Shop,Clothing Store,Spa,Furniture / Home Store,Fast Food Restaurant,Mexican Restaurant,Miscellaneous Shop,Diner,Dessert Shop,Park
474,43.715383,-79.400304,Salon / Barbershop,3,Coffee Shop,Clothing Store,Spa,Furniture / Home Store,Fast Food Restaurant,Mexican Restaurant,Miscellaneous Shop,Diner,Dessert Shop,Park
475,43.715383,-79.400653,Spa,3,Coffee Shop,Clothing Store,Spa,Furniture / Home Store,Fast Food Restaurant,Mexican Restaurant,Miscellaneous Shop,Diner,Dessert Shop,Park
476,43.715383,-79.400406,Chinese Restaurant,3,Coffee Shop,Clothing Store,Spa,Furniture / Home Store,Fast Food Restaurant,Mexican Restaurant,Miscellaneous Shop,Diner,Dessert Shop,Park
477,43.715383,-79.400283,Dessert Shop,3,Coffee Shop,Clothing Store,Spa,Furniture / Home Store,Fast Food Restaurant,Mexican Restaurant,Miscellaneous Shop,Diner,Dessert Shop,Park
478,43.715383,-79.400017,Fast Food Restaurant,3,Coffee Shop,Clothing Store,Spa,Furniture / Home Store,Fast Food Restaurant,Mexican Restaurant,Miscellaneous Shop,Diner,Dessert Shop,Park
479,43.715383,-79.399776,Coffee Shop,3,Coffee Shop,Clothing Store,Spa,Furniture / Home Store,Fast Food Restaurant,Mexican Restaurant,Miscellaneous Shop,Diner,Dessert Shop,Park


Cluster 5 - Fast Food Restaurants, Liquor Store, and Restaurants

In [100]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2173,43.605647,-79.499302,Liquor Store,4,Fast Food Restaurant,Liquor Store,Restaurant,Coffee Shop,Sandwich Place,Seafood Restaurant,Café,Pizza Place,Flower Shop,Pharmacy
2174,43.605647,-79.500908,Pizza Place,4,Fast Food Restaurant,Liquor Store,Restaurant,Coffee Shop,Sandwich Place,Seafood Restaurant,Café,Pizza Place,Flower Shop,Pharmacy
2175,43.605647,-79.503281,Restaurant,4,Fast Food Restaurant,Liquor Store,Restaurant,Coffee Shop,Sandwich Place,Seafood Restaurant,Café,Pizza Place,Flower Shop,Pharmacy
2176,43.605647,-79.503012,Bakery,4,Fast Food Restaurant,Liquor Store,Restaurant,Coffee Shop,Sandwich Place,Seafood Restaurant,Café,Pizza Place,Flower Shop,Pharmacy
2177,43.605647,-79.503056,Café,4,Fast Food Restaurant,Liquor Store,Restaurant,Coffee Shop,Sandwich Place,Seafood Restaurant,Café,Pizza Place,Flower Shop,Pharmacy
2178,43.605647,-79.498275,Sandwich Place,4,Fast Food Restaurant,Liquor Store,Restaurant,Coffee Shop,Sandwich Place,Seafood Restaurant,Café,Pizza Place,Flower Shop,Pharmacy
2179,43.605647,-79.498963,Fast Food Restaurant,4,Fast Food Restaurant,Liquor Store,Restaurant,Coffee Shop,Sandwich Place,Seafood Restaurant,Café,Pizza Place,Flower Shop,Pharmacy
2180,43.605647,-79.499400,Fried Chicken Joint,4,Fast Food Restaurant,Liquor Store,Restaurant,Coffee Shop,Sandwich Place,Seafood Restaurant,Café,Pizza Place,Flower Shop,Pharmacy
2181,43.605647,-79.502164,Pharmacy,4,Fast Food Restaurant,Liquor Store,Restaurant,Coffee Shop,Sandwich Place,Seafood Restaurant,Café,Pizza Place,Flower Shop,Pharmacy
2182,43.605647,-79.501147,Seafood Restaurant,4,Fast Food Restaurant,Liquor Store,Restaurant,Coffee Shop,Sandwich Place,Seafood Restaurant,Café,Pizza Place,Flower Shop,Pharmacy
